In [ ]:
from floweaver import *
import pandas as pd
import numpy as np

In [3]:
flows = pd.read_excel('glass16.xlsx', sheetname='flows')
processes = pd.read_excel('glass16.xlsx', sheetname='processes', index_col=0)
dataset = Dataset(flows, processes)
flows

,source,target,value,uncertainty
0,Soda Ash,FG Batch,18.208757,4
1,Silica,FG Batch,51.618435,4
2,Limestone,FG Batch,11.433241,4
3,CG Cullet,CG Batch,29.669500,4
4,Soda Ash,CG Batch,14.606371,4
5,Silica,CG Batch,41.406341,4
6,Limestone,CG Batch,9.171310,4
7,CG Batch,Int CG,94.188889,3
8,CG Batch,CG Process Emissions,10.083522,4
9,FG Batch,Int FG,80.811765,3


In [4]:
partition_prep = Partition.Simple('process', ['Silica', 'Soda Ash', 'Limestone','CG Cullet'])
partition_middle = Partition.Simple('process', ['CG Batch'])
partition_mi = Partition.Simple('process', ['FG Batch'])
partition_tres = Partition.Simple('process', ['Int CG','CG Process Emissions'])
partition_tr = Partition.Simple('process', ['Int FG','FG Process Emissions'])
partition_quat = Partition.Simple('process', ['Final FG', 'Final CG'])
partition_cinq = Partition.Simple('process', ['Buildings', 'Automotive','Solar','Other FG','Beverages','Food','Other CG'])
partition_six = Partition.Simple('process', ['Landfill'])

nodes = {
    'Batch Prep': ProcessGroup('group == "Batch preparation"',
                               partition_prep,title="Batch prep"),
    'Middle': ProcessGroup('group == "Premelt CG"',
                           partition_middle,title="Premelt CG"),
      'Mi': ProcessGroup('group == "Premelt FG"',
                           partition_mi,title="Premelt FG"),
    'Tres': ProcessGroup('group == "Postmelt CG"',
                           partition_tres,title="Postmelt CG"),
        'Tr': ProcessGroup('group == "Postmelt FG"',
                           partition_tr,title="Postmelt FG"),
     'Quat': ProcessGroup('group == "Postcheck"',
                           partition_quat,title="Postcheck"),
    'Cinq': ProcessGroup('group == "End-use"',
                           partition_cinq,title="End-use"),
    'Six': ProcessGroup('group == "End-of-Life"',
                           partition_six,title="End-of-Life"),
    # add more layers...
}

bundles = [
    Bundle('Batch Prep', 'Middle'),
    Bundle('Batch Prep', 'Mi'),
    Bundle('Middle', 'Tres'),
    Bundle('Mi', 'Tr'),
    Bundle('Tres', 'Middle'),
    Bundle('Tr', 'Mi'),
    Bundle('Tres','Quat'),
    Bundle('Tr','Quat'),
    Bundle('Quat','Cinq'),
    Bundle('Cinq','Six'),
    Bundle('Cinq','Batch Prep')
    # add more bundles between layers...
]

ordering = [
    ['Batch Prep'],
    ['Mi','Middle'],
    ['Tr','Tres'],
    ['Quat'],
    ['Cinq'],
    ['Six']
    # add more layers...
]

sdd = SankeyDefinition(nodes, bundles, ordering,flow_partition=Partition.Simple('uncertainty', [1, 2, 3, 4, 5]))
# First let's display the Sankey
weave(sdd, dataset,palette='Paired_12').to_widget().auto_save_png('glassfinal.png')

A Jupyter Widget

In [5]:
#Now let's save the Sankey as a json file, in order to be able to edit it in Captain Sankey and later on in Inkscape
weave(sdd, dataset,palette='Paired_12').to_json('editsankey.json')

In [6]:
run fixup_sankey_json.py editsankey.json editsankey_fixed.json

In [7]:
#Now with uncertainty
scale = QuantitativeScale('uncertainty', palette='Blues_9')
weave(sdd, dataset,measures={'value': 'sum', 'uncertainty': 'mean'},link_width='value',link_color=scale).to_widget().auto_save_png('glassuncert.png')

A Jupyter Widget

In [ ]:
#Same steps as above to get a json file
scale = QuantitativeScale('uncertainty', palette='Blues_9')
weave(sdd, dataset,measures={'value': 'sum', 'uncertainty': 'mean'},link_width='value',link_color=scale).to_json('editsankey1.json')

In [ ]:
run fixup_sankey_json.py editsankey1.json editsankey1_fixed.json